# 1. Generating the dataset

This notebook will be dedicated to loading the data we've already exported on [this notebook](./retrieving_the_data.ipynb).

So, in order to generate the dataset, let's load that up first

## 1.a. if you extracted the data from Twitter's API

In [20]:
import json

with open('data/data.json', 'r') as f:
    data = json.load(f)

Now we've got the data in a dict called `data`, and we can start crafting our dataset.

First, let's explore our data to see what we're interested in

In [8]:
data[1]

{'quote_count': 1,
 'contributors': None,
 'truncated': True,
 'text': '@yimitorres554 @IvanDuque Que es ser un seguidor de Petro: Ser una persona carente de inteligencia, respeto .....\nS… https://t.co/yjIzCekQB5',
 'is_quote_status': False,
 'in_reply_to_status_id': 968212447197884417,
 'reply_count': 1,
 'id': 968274349840060416,
 'favorite_count': 1,
 'entities': {'user_mentions': [{'id': 77653794,
    'indices': [15, 25],
    'id_str': '77653794',
    'screen_name': 'IvanDuque',
    'name': 'Iván Duque'}],
  'symbols': [],
  'hashtags': [],
  'urls': [{'url': 'https://t.co/yjIzCekQB5',
    'indices': [117, 140],
    'expanded_url': 'https://twitter.com/i/web/status/968274349840060416',
    'display_url': 'twitter.com/i/web/status/9…'}]},
 'retweeted': False,
 'coordinates': None,
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_screen_name': 'yimitorres35',
 'id_str': '968274349840060416',
 'display_text_range': [26, 140],
 'retwe

With this, we can identify the attributes we're interested in. Namely:

* `created_at` - This is the date the tweet was created at.
* `favorite_count` - Pretty self explaining. The number of times the tweet was marked as favorite.
* `source` - The app through which the tweet was created. e.g. 'Twitter for Android'.
* `text*` - The text of the tweet. The most important thing we're after.
* `user['id']` - We'll keep the user's id to be able to identify changes

In [13]:
import pandas as pd
import numpy as np

# First we create a list of the tweet text, taking the full text for tweets that were truncated
contents = [t['extended_tweet']['full_text'] if t['truncated'] else t['text'] for t in data]

# And we generate a dataframe with just the text, and add each of the columns we're interested in

df = pd.DataFrame(data = contents, columns=['text'])

df.head()

,text
0,@t_hendel @IvanDuque Yo creo que todo no fué n...
1,@yimitorres554 @IvanDuque Que es ser un seguid...
2,@IvanDuque https://t.co/ikwqiglu2g
3,El presidente @AlvaroUribeVel y su candidato @...
4,@IvanDuque Asegura bienestar a los residentes ...


In [21]:
# And we now add the columns. We'll use Numpy arrays so that the datatype can be inferred
df['date'] = np.array([t['created_at'] for t in data])
df['favorited'] = np.array([t['favorite_count'] for t in data])
df['retweeted'] = np.array([t['retweet_count'] for t in data])
df['replies'] = np.array([t['reply_count'] for t in data])
df['user'] = np.array(t['user']['id'] for t in data])

df.head()

,text,date,favorited,retweeted,replies
0,@t_hendel @IvanDuque Yo creo que todo no fué n...,Mon Feb 26 23:59:35 +0000 2018,0,0,0
1,@yimitorres554 @IvanDuque Que es ser un seguid...,Mon Feb 26 23:59:29 +0000 2018,1,0,1
2,@IvanDuque https://t.co/ikwqiglu2g,Mon Feb 26 23:59:26 +0000 2018,0,0,0
3,El presidente @AlvaroUribeVel y su candidato @...,Mon Feb 26 23:59:01 +0000 2018,0,0,0
4,@IvanDuque Asegura bienestar a los residentes ...,Mon Feb 26 23:58:52 +0000 2018,0,0,0


## 1.b. If you extracted the data using the extractor


In [4]:
import json

with open('data/scraped_data2.json', 'r') as f:
    data = json.load(f)

In [27]:
data[0]

{'text': 'Los "pollos" queremos un Presidente "pollo". Un pollo fresco e innovador pero con las espuelas puestas por @AlvaroUribeVel Ese "pollo" es @IvanDuque #DuquePresidente @CeDemocratico @CarlosHolmesTru',
 'created_at': '1517091885',
 'lang': 'es',
 'user': '708108228568207360',
 'replies': '85',
 'retweeted': '336',
 'favorited': '629'}

In [40]:
import numpy as np
import pandas as pd

# And we generate a dataframe with just the text, and add each of the columns we're interested in

contents = [t['text'] for t in data]

df = pd.DataFrame(data = contents, columns=['text'])

df['date'] = np.array([int(t['created_at']) for t in data])
df['favorited'] = np.array([int(t['favorited']) for t in data])
df['retweeted'] = np.array([int(t['retweeted']) for t in data])
df['replies'] = np.array([int(t['replies']) for t in data])
df['user'] = np.array([int(t['user']) for t in data])
df['lang'] = np.array([t['lang'] for t in data])

df.head()

,text,date,favorited,retweeted,replies,user,lang
0,"Los ""pollos"" queremos un Presidente ""pollo"". U...",1517091885,629,336,85,708108228568207360,es
1,Vota por @IvanDuque en la consulta @CeDemocrat...,1517089885,793,535,136,149281495,es
2,Compartimos con alegría nuestra propuesta de p...,1517089815,188,119,4,77653794,es
3,.@FNAraujoR #4 Senado @IvanDuque #ElCandidato...,1517089573,34,27,2,1069678676,es
4,The girls flocking to see Mr. Duque. That's g...,1517086450,1,0,0,876674787115925504,en


And that's it. We're now ready to start cleaning and analyzing the data

## 2. Cleaning the data

We've now got a working dataframe. However, the `text` column has some problems (links at the end of the tweet are attached to the last word). And the date column is there, but it isn't really recognized as a date yet.

So, first, let's convert the timestamps in the date column into datetimes, so that we can then use the date as our index

In [41]:
# We can use pd.to_datetime, using the unit argument, to indicate that our timestamps are
# measured in seconds 
df['date'] = pd.to_datetime(df['date'], unit='s')

In [42]:
df['date']

0       2018-01-27 22:24:45
1       2018-01-27 21:51:25
2       2018-01-27 21:50:15
3       2018-01-27 21:46:13
4       2018-01-27 20:54:10
5       2018-01-27 20:44:49
6       2018-01-27 19:48:22
7       2018-01-27 19:11:12
8       2018-01-27 18:57:43
9       2018-01-27 18:05:48
10      2018-01-27 17:53:29
11      2018-01-27 17:51:35
12      2018-01-27 17:07:42
13      2018-01-27 17:05:10
14      2018-01-27 16:31:51
15      2018-01-27 15:20:26
16      2018-01-27 15:03:18
17      2018-01-27 14:54:48
18      2018-01-27 13:55:25
19      2018-01-27 22:59:15
20      2018-01-27 22:48:21
21      2018-01-27 22:08:44
22      2018-01-27 21:57:34
23      2018-01-27 20:35:02
24      2018-01-27 20:14:42
25      2018-01-27 18:44:33
26      2018-01-27 18:04:49
27      2018-01-27 17:31:39
28      2018-01-27 17:28:25
29      2018-01-27 16:53:41
                ...        
27665   2018-06-17 23:42:05
27666   2018-06-17 23:41:46
27667   2018-06-17 23:41:24
27668   2018-06-17 23:41:14
27669   2018-06-17 2

As you can see, we now have dates we can understand.

Now, let's examine the text of some tweets so that we can clean them up

In [102]:
for text in df['text'][0:50]:
    print(repr(text))
    print("\n", "=" * 70, "\n")

'Los "pollos" queremos un Presidente "pollo". Un pollo fresco e innovador pero con las espuelas puestas por @AlvaroUribeVel Ese "pollo" es @IvanDuque #DuquePresidente @CeDemocratico @CarlosHolmesTru'


'Vota por @IvanDuque en la consulta @CeDemocratico el partido del Uribismo y de @AlvaroUribeVelpic.twitter.com/7CzVECAOHu'


'Compartimos con alegría nuestra propuesta de país en #Medellín, juntos vamos a construir un país de #legalidad #emprendimiento y #equidad #ManoFirme #CorazónGrandepic.twitter.com/7JZKfbxiYR'


'.@FNAraujoR  #4 Senado @IvanDuque #ElCandidatoDeLaEsperanza  #1 en la Consulta Interpartidista @AlvaroUribeVel  # 1 Senado .@CeDemocratico , están sembrando Esperanza en cada Rincón de La Patria para que los tiempos de la Seguridad Democrática Regrese. #LoMejorEstaPorVenirpic.twitter.com/RWrmqrwYfm'


"The girls flocking to see Mr. Duque.  That's good."


'Nuestro gobierno incentivará la inversión y el emprendimiento, buscará, con una agenda integral de recuperación económi

We can identify a couple of issues. The first one, is that there line breaks, were replaced by `"\n"`, which is still line breaks and which will output nicely when we print the strings, but might give us trouble when processing. So the first thing we do is removing that.

In [115]:
# We'll replace the line breaks by spaces
df['text'] = df['text'].apply(lambda x: x.replace("\n", " "))

There's also an issue with characters that look like spaces, but use a different encoding. And we want them to behave as spaces so that we can split taking those into account. So, we replace the line from above for this:

In [141]:
df['text'] = df['text'].apply(lambda x: x.replace("\n", " ").replace("\xa0", " "))

That leaves us with one fewer problem.

Now, we can see that some tweets, like this one:

> Vota por @IvanDuque en la consulta @CeDemocratico el partido del Uribismo y de ** @AlvaroUribeVelpic.twitter.com/7CzVECAOHu**

and this one:

> Compartimos con alegría nuestra propuesta de país en #Medellín, juntos vamos a construir un país de #legalidad #emprendimiento y #equidad #ManoFirme **#CorazónGrandepic.twitter.com/7JZKfbxiYR**

Have the same issue. They have a link at the end, which is attached to the last word of the tweet. This is because when we extracted the data, we used BeautifulSoup's `text` function. Which returns the text contents of an item, so for example if it found something like

```html
<div class="tweet">
    <p class="tweet-text">Compartimos con alegría nuestra propuesta de país en #Medellín, juntos vamos a construir un país de #legalidad #emprendimiento y #equidad #ManoFirme #CorazónGrande</p>
    <a class="attached-media" href="https://pic.twitter.com/7JZKfbxiYR"><img src="https://pic.twitter.com/7JZKfbxiYR" alt="pic.twitter.com/7JZKfbxiYR"/></a>
</div>
```

This is not really extracted from Twitter, it's just to make a point. So, here, if we found the div with class `tweet` and then extracted its `text` attribute, it would give us the text between the `<p>` tags, and then the `alt` text of the `img` tag, but since there are no line breaks or anything really separating the `<p>` tag from the `<img>` one, the text will become attached.

So, to solve this issue, let's first identify the different types of links that might have become attached to the text of the tweets

In [142]:
# This will give us the last "words" in the tweets
last_words = df['text'].apply(lambda x: x.split(" ")[-1])

# This will filter those that contain .com, .be, .ly or http. We might still be missing some ones here
with_links = last_words[last_words.str.contains('http|\.com|\.be|\.ly|\.me', regex=True)]

In [143]:
with_links

1                @AlvaroUribeVelpic.twitter.com/7CzVECAOHu
2                 #CorazónGrandepic.twitter.com/7JZKfbxiYR
3           #LoMejorEstaPorVenirpic.twitter.com/RWrmqrwYfm
5                      #Medellínpic.twitter.com/DFEW7027qU
6                     #Antioquiapic.twitter.com/BccKObHfib
7                               pic.twitter.com/yed48GqJ0B
8                      #Medellínpic.twitter.com/lFyaU9zDaa
10                     #Medellínpic.twitter.com/oXtjIjIC8R
11                              pic.twitter.com/Fk3R1ZW7dW
12                              pic.twitter.com/0Bq0ynfaje
14       #LaFuerzaUribistaDelVallepic.twitter.com/Qzc8R...
16                  @natiibedoyapic.twitter.com/0bRMbTiTTp
18                 @simonmolinagpic.twitter.com/RIX23JfxjF
19                    @jcjurado1pic.twitter.com/aO4yoc9wan
24          #DuquePresidente2018pic.twitter.com/1d3hk5Pl11
26               #VolvamosACreerpic.twitter.com/94z0z8V58T
28                    #VotaCD102pic.twitter.com/M90AA9ZD

In [144]:
len(with_links)

13401

As you may see, almost half of our tweets end in links. However, not all of them were attached to words, that is, some of them are just links. So, those don't need to be modified. Therefore, we'll take them out of our list

In [184]:
matches = with_links[~with_links.str.startswith(("http", "bit", "fb", "youtu", "pic", "twitter"))]

In [185]:
matches

1                @AlvaroUribeVelpic.twitter.com/7CzVECAOHu
2                 #CorazónGrandepic.twitter.com/7JZKfbxiYR
3           #LoMejorEstaPorVenirpic.twitter.com/RWrmqrwYfm
5                      #Medellínpic.twitter.com/DFEW7027qU
6                     #Antioquiapic.twitter.com/BccKObHfib
8                      #Medellínpic.twitter.com/lFyaU9zDaa
10                     #Medellínpic.twitter.com/oXtjIjIC8R
14       #LaFuerzaUribistaDelVallepic.twitter.com/Qzc8R...
16                  @natiibedoyapic.twitter.com/0bRMbTiTTp
18                 @simonmolinagpic.twitter.com/RIX23JfxjF
19                    @jcjurado1pic.twitter.com/aO4yoc9wan
24          #DuquePresidente2018pic.twitter.com/1d3hk5Pl11
26               #VolvamosACreerpic.twitter.com/94z0z8V58T
28                    #VotaCD102pic.twitter.com/M90AA9ZDJF
29               @AlvaroUribeVelpic.twitter.com/ccc1ZyZC7N
31               @AlvaroUribeVelpic.twitter.com/9tZ2ScyVCT
36              #DuquePresidentepic.twitter.com/OPA3dGj9

and that leaves us with only those links that became attached to the last word of their tweets.